## Interrupt

This notebook assumes knowledge of what double-texting is, which you can learn about in the [double-texting conceptual guide](https://langchain-ai.github.io/langgraph/cloud/concepts/#double-texting).

The guide covers the `interrupt` option for stopping double texting, which interrupts the prior run of the graph and starts a new one with the double-text. This option does not delete the first run, but rather keeps it in the database but sets its status to `interrupted`. Below is a quick example of using the `interrupt` option.

First, let's import our required packages and instantiate our client, assistant, and thread.

In [185]:
import asyncio

from langchain_core.messages import convert_to_messages
from langgraph_sdk import get_client

In [186]:
client = get_client()
assistant_id = "agent"
thread = await client.threads.create()

In [187]:
# the first run will be interrupted
interrupted_run = await client.runs.create(
    thread["thread_id"],
    assistant_id,
    input={"messages": [{"role": "human", "content": "what's the weather in sf?"}]},
)
await asyncio.sleep(2)
run = await client.runs.create(
    thread["thread_id"],
    assistant_id,
    input={"messages": [{"role": "human", "content": "what's the weather in nyc?"}]},
    multitask_strategy="interrupt",
)

In [188]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], run["run_id"])

We can see that the thread has partial data from the first run + data from the second run

In [189]:
state = await client.threads.get_state(thread["thread_id"])

In [190]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

what's the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_vRSI26INZ2CD3CQrVtM6kzVm)
 Call ID: call_vRSI26INZ2CD3CQrVtM6kzVm
  Args:
    query: current weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.wunderground.com/hourly/us/ca/san-francisco/date/2024-6-25", "content": "San Francisco Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & tropical weather conditions for the San Francisco area. ... Tuesday 06/25 ..."}, {"url": "https://forecast.weather.gov/zipcity.php?inputstring=San+Francisco,CA", "content": "San Francisco CA 37.77\u00b0N 122.41\u00b0W (Elev. 131 ft) Last Update: 5:02 pm PDT Jun 23, 2024. Forecast Valid: 5pm PDT Jun 23, 2024-6pm 

Verify that the original, interrupted run was interrupted

In [191]:
(await client.runs.get(thread["thread_id"], interrupted_run["run_id"]))["status"]

'interrupted'